In [16]:
import pandas as pd
import statistics
import re
from datetime import datetime
from collections import defaultdict
import json

# Lab2 - Data Collection and Pre-processing

## 1. Hello, Data!

In [17]:
#Load raw CSV, display first 3 rows
df_transactions = pd.read_csv('data/ecommerce_dataset.csv')
df_transactions.head(3)

,date,customer_id,product,price,quantity,coupon_code,shipping_city
0,2023-05-15,fc1fd698-529d-4978-8196-2a4177424066,Noise Cancelling Earbuds,138.52,5,FREESHIP05,Rochamouth
1,2024-04-22,8fa43fff-c1cf-4744-9de9-2df45832a513,Laptop Stand,115.27,4,SAVE10,East David
2,2024-01-15,11155053-7973-48a4-9fd0-56546744e9e1,Portable Charger,476.25,1,SAVE10,South Michealland


## 2. Pick the Right Container
Dictionary, namedtuple and class all have their own pros and cons. 
* Dicionary is mutalbe, lightwighted, and easy to serialize/deserialize
* Namedtuple is readable, fast, lightweighted, and immutalble
* Class is used to encapsulate data and logic. It is customizable and flexible. By default it's mutalbe.

Based on what we are going to do in this lab, class is the best choice. It can encapsulate the clean logic and all the transaction data. Future engineering property (days_since_purchase) can be also easily added as a calculated field.

## 3. Transaction Class and OO data structure

In [18]:
#Transaction class
class Transaction:
    def __init__(self, list):
        self._date = datetime.strptime(list[0], "%Y-%m-%d")
        self._customerId = list[1]
        self._product_name = list[2]
        self._price = list[3]
        self._quantity = list[4]
        self._couponCode = list[5]
        self._shipToCity = list[6]

    @property
    def Date(self):
        return self._date

    @Date.setter
    def Date(self, value):
        self._date = value

    @property
    def ProductName(self):
        return self._product_name

    @ProductName.setter
    def ProductName(self, value):
        self._product_name = value

    @property
    def Price(self):
        return self._price

    @Price.setter
    def Price(self, value):
        self._price = value

    @property
    def Quantity(self):
        return self._quantity

    @Quantity.setter
    def Quantity(self, value):
        self._quantity = value
    
    @property
    def CustomerId(self):
        return self._customerId
    
    @CustomerId.setter
    def CustomerId(self, value):
        self._customerId = value

    @property
    def ShipToCity(self):
        return self._shipToCity
    
    @ShipToCity.setter
    def ShipToCity(self, value):
        self._shipToCity = value

    @property
    def Discount(self):
        if (str(self._couponCode) == 'nan'):
            return 0
        match = re.search(r'(\d{2})$', self._couponCode)
        if match:
            return int(match.group(1))
        else:
            return 0

    @property
    def DaysSincePurchase(self):
        return (datetime.today() - self.Date).days

    def ToString(self):
        return f"Date: {self.Date.strftime("%Y-%m-%d")}, CustomerId: {self.CustomerId}, ProductName: {self.ProductName}, Price: {self.Price}, Quantity: {self.Quantity}, Ship to City: {self.ShipToCity}"

## 4. Bulk Loader

In [19]:
class TransactionDataLoader:
    #load_transactions() returning list of transactions
    def load_transactions(self, df):
        transactionList = []
        for _, row in df.iterrows():
            transaction = Transaction(row.tolist())
            transactionList.append(transaction)
        
        return transactionList
    
# load and display transactions
transactions = TransactionDataLoader().load_transactions(df_transactions)
for t in transactions[:3]:
    print(t.ToString())

Date: 2023-05-15, CustomerId: fc1fd698-529d-4978-8196-2a4177424066, ProductName: Noise Cancelling Earbuds, Price: 138.52, Quantity: 5, Ship to City: Rochamouth
Date: 2024-04-22, CustomerId: 8fa43fff-c1cf-4744-9de9-2df45832a513, ProductName: Laptop Stand, Price: 115.27, Quantity: 4, Ship to City: East David
Date: 2024-01-15, CustomerId: 11155053-7973-48a4-9fd0-56546744e9e1, ProductName: Portable Charger, Price: 476.25, Quantity: 1, Ship to City: South Michealland


## 5. Quick Profiling

In [20]:
#calculate min/mean/max price, and unique city count 
class DataProfiler:
    def __init__(self, transactions):
        self._transactionList = transactions
    def minPrice(self):
        return min(t.Price for t in self._transactionList)
    
    def maxPrice(self):
        return max(t.Price for t in self._transactionList) 
    
    def meanPrice(self):
        return statistics.mean(t.Price for t in self._transactionList)

    def uniqueCityCount(self):
        return len(set(t.ShipToCity for t in self._transactionList))

#calculate and display prifiling
dataProfiler = DataProfiler(transactions)
min_price = dataProfiler.minPrice()
max_price = dataProfiler.maxPrice()
mean_price = dataProfiler.meanPrice()
uniqueCityCount = dataProfiler.uniqueCityCount()

print(f'Min price: {min_price}')
print(f'Max price: {max_price}')
print(f'Mean price: {mean_price}')
print(f'Unique country count: {uniqueCityCount}')

Min price: -24.78
Max price: 499.92
Mean price: 253.20124
Unique country count: 51


## 6. Spot the Grime
* There is a transaction with negative price
* There is a transaction with negative quantity
* There is a transaction with ship to city as "N/A"

## 7. Cleaning Rules

In [21]:
# Find the Grime, and clean
class DataCleaner:
    def __init__(self, transactions):
        self._transactions = transactions

    def clean(self):
        #find and remove transactions with negative price
        negativePrices = [t for t in self._transactions if t.Price < 0]
        for t in negativePrices:
            self._transactions.remove(t)

        #find and remove transactions with negative quantity
        negativeQuantities = [t for t in self._transactions if t.Quantity < 0]
        for t in negativeQuantities:
            self._transactions.remove(t)

        #standardise invalid city to None
        invalidCities = [t for t in self._transactions if str(t.ShipToCity) == 'nan']
        for t in invalidCities:
            t.Country = 'None'

#show “before/after” counts
print(f"Before count: {len(transactions)}")
dataCleaner = DataCleaner(transactions).clean()
print(f"After count: {len(transactions)}")

Before count: 500
After count: 498


## 8. Transformations
* Parse coupon_code ➞ numeric discount: added field (Discount) to Transaction class

In [22]:
#Use regular expression to get the numeric discount from the coupon code
value = "SAVEH20"
match = re.search(r'(\d{2})$', value)
if match:
    last_two_digits = int(match.group(1))

#show the Discount of the top 3 transactions
for t in transactions[:5]:
    print(t.Discount)

5
10
10
5
0


## 9. Feature Engineering
Add field DaysSincePurchase to class Transaction

In [23]:
#show the DaysSincePurchase of the top 3 transactions
for t in transactions[:3]:
    print(t.DaysSincePurchase)

742
399
497


## 10. Mini-Aggregation

In [24]:
class Aggregator:
    def __init__(self, transactions):
        self._transactions = transactions

    #calculate revenue per shipping_city
    def calculateRevenuePerCity(self):
        revenue_by_city = defaultdict(float)
        for t in self._transactions:
            revenue_by_city[t.ShipToCity] += float(t.Price * t.Quantity * (100 - t.Discount))

        return revenue_by_city

#display result
revenues = Aggregator(transactions).calculateRevenuePerCity()
for city, total in revenues.items():
    print(f"{city}: {total:,.2f}")

Rochamouth: 1,157,409.45
East David: 920,150.70
South Michealland: 1,415,248.40
Harmonport: 685,554.05
North Beth: 572,158.70
South Jonathanhaven: 840,471.75
East Susan: 1,094,838.20
Ericaside: 1,269,520.95
West Raymondport: 351,901.40
Smithshire: 775,639.70
North John: 618,924.30
East Debrahaven: 305,796.90
Beltranfort: 1,048,049.20
Danielbury: 542,227.30
nan: 62,910.00
South Dana: 692,156.70
Barryberg: 408,847.60
North Nicoletown: 731,266.00
Port Corey: 1,029,509.75
East Jason: 571,066.40
New Jane: 845,271.55
North Jaredshire: 1,012,195.00
Stevenport: 1,032,269.10
New Melissa: 504,117.05
Desireestad: 706,697.70
Harrischester: 462,824.00
East Shannon: 765,188.40
Rodriguezborough: 1,149,738.60
Jensenborough: 482,648.10
Stevenshire: 604,372.70
North Robert: 675,346.85
Lanefurt: 600,500.20
Millerchester: 272,849.00
Dalefurt: 567,961.05
West Nicoleburgh: 605,853.00
Manuelview: 991,420.15
Melissaburgh: 957,158.60
North Kevin: 541,633.50
Lauramouth: 841,872.30
East Michellemouth: 538,548.40

## 11. Serialization Checkpoint

In [25]:
# Serialization 
class Serializer:
    def __init__(self, transactions):
        self._transactions = transactions
        self._transactions_dict = [
            {
                "Date": t.Date.strftime("%Y-%m-%d"),
                "CustomerId": t.CustomerId,
                "ProductName": t.ProductName,
                "Price": float(t.Price),
                "Discount": t.Discount,
                "Quantity": t.Quantity,
                "Shit to City": t.ShipToCity
            }
            for t in transactions
        ]
    #Save cleaned data to Json
    def saveToJson(self, filePath):
        with open(filePath, "w") as f:
            json.dump(self._transactions_dict, f, indent=4)
    #Save cleaned data to Parquet
    def saveToParquet(self, filePath):
        df = pd.DataFrame(self._transactions_dict)
        df.to_parquet(filePath, index=False)

#Save cleaned data
serializer = Serializer(transactions)
serializer.saveToJson('artifacts/transaction_json.json')
serializer.saveToParquet('artifacts/transaction_parquet.parquet')

## 12. Soft Interview Reflection -  how OOP helped
In this lab, OOP helps encapsulate all the properties (fields) of a transaction into an object, which represents a row in the transaction dataset. Since the object is mutable, it makes data cleaning and transformation easier. It also allows encapsulating logic within the object—such as extracting a numeric discount from a coupon code or calculating the number of days since purchase. Furthermore, using objects makes it easier for humans to understand the actions, as they resemble our daily interactions with real-life objects.

## Data-Dictionary Section

In [26]:
from IPython.display import Markdown
#convert transaction dataframe to dict
dict_transaction = df_transactions.to_dict(orient='records')
#read product catalog data
df_product = pd.read_csv('data/product_catalog.csv')
#convert dataframe to a data dictionary with product name as key
product_dict = {item['product_name']: {k: v for k, v in item.items() if k != 'product_name'} for _, item in df_product.iterrows()}
#merge product info to transactions
for t in dict_transaction:
    product_info = product_dict.get(t['product'], {})  # Use .get to avoid KeyError
    t.update(product_info)
df = pd.DataFrame(dict_transaction)
# Display as markdown table (first 5 rows for brevity)
table = df.head().to_markdown(index=False)
display(Markdown(table))


| date       | customer_id                          | product                  |   price |   quantity | coupon_code   | shipping_city     | product_id                           | category    |   stock | supplier                       |   rating |
|:-----------|:-------------------------------------|:-------------------------|--------:|-----------:|:--------------|:------------------|:-------------------------------------|:------------|--------:|:-------------------------------|---------:|
| 2023-05-15 | fc1fd698-529d-4978-8196-2a4177424066 | Noise Cancelling Earbuds |  138.52 |          5 | FREESHIP05    | Rochamouth        | c59139a8-8a2d-43f2-a1a5-ec6a5b5caa85 | Accessories |     202 | Anderson, Holloway and Bennett |      5   |
| 2024-04-22 | 8fa43fff-c1cf-4744-9de9-2df45832a513 | Laptop Stand             |  115.27 |          4 | SAVE10        | East David        | 37a9630a-cfa3-49bd-ba09-be80ce736e78 | Peripherals |     117 | Johnson, Weber and White       |      3.1 |
| 2024-01-15 | 11155053-7973-48a4-9fd0-56546744e9e1 | Portable Charger         |  476.25 |          1 | SAVE10        | South Michealland | e315b3dc-b69f-4307-80f9-bd637b9966f3 | Peripherals |     311 | Harris-Lang                    |      4.7 |
| 2023-06-12 | 32aaf677-3403-4230-97ad-651ec2b74d24 | Gaming Keyboard          |   24.79 |          1 | FREESHIP05    | Harmonport        | 88223c05-783f-44bb-a2ae-392e29e61185 | Peripherals |      97 | Thomas and Sons                |      4.9 |
| 2023-01-11 | d08ded6e-d60f-44e5-9d88-7db65d774c60 | LED Monitor              |  254.36 |          3 | nan           | North Beth        | 2e8c42ec-5457-4e8c-b828-5b5c3177b298 | Peripherals |     403 | Liu PLC                        |      3.3 |